## 5. MLP

In [3]:
#!pip install scikit-optimize
#!pip install optuna
#!pip install pycaret
#from pycaret.classification import *
import pandas as pd                                             #Importar libreria pandas para leer archivo parquet
pd.set_option('display.float_format', '{:.2f}'.format)          #Formatro decimal para el display de los datos
from scipy.stats import skew, kurtosis, chi2_contingency          #Importar libreria scipy para calcular skew y kurtosis
import seaborn as sns                                           #Importar libreria seaborn para graficar
from sklearn.model_selection import train_test_split, RandomizedSearchCV            #Importar libreria sklearn para separar datos en train y test
from sklearn.dummy import DummyClassifier                       #Importar libreria sklearn para crear clasificador dummy
from sklearn.metrics import roc_auc_score, roc_curve, auc       #Importar libreria sklearn para calcular metricas de clasificacion binaria
import matplotlib.pyplot as plt                                 #Importar libreria matplotlib para graficar
import numpy as np                                              #Importar libreria numpy para calculos matematicos
from sklearn.pipeline import Pipeline                           #Importar libreria sklearn para crear pipelines
from sklearn.base import BaseEstimator, TransformerMixin        #La clase ColumnSelector hereda de BaseEstimator y TransformerMixin, lo que permite integrarse fácilmente en un pipeline de scikit-learn
from sklearn.impute import SimpleImputer                        #Importar libreria sklearn para imputar valores faltantes
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler, RobustScaler #Importar libreria sklearn para escalar y codificar variables
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression             #Importar libreria sklearn para crear modelo de regresion logistica
from sklearn.svm import SVC                                     #Importar libreria sklearn para crear modelo de SVM
from skopt import BayesSearchCV                                 #Importar libreria skopt para optimizar hiperparametros
from skopt.space import Real, Integer                           #Importar libreria skopt para definir espacio de busqueda de hiperparametros
import optuna                                                   #Importar libreria optuna para optimizar hiperparametros
from sklearn.ensemble import RandomForestClassifier             #Importar libreria sklearn para crear modelo de Random Forest
from sklearn.model_selection import train_test_split            #
from sklearn.ensemble import RandomForestClassifier             #
from sklearn.neural_network import MLPClassifier                #Importar libreria sklearn para crear modelo de redes neuronales

In [4]:
#Vista general del conjunnto de datos de entrenamiento
train_dataset = pd.read_parquet ("df_train.parquet")
data_df = pd.read_parquet ("df_train.parquet")

In [5]:
#Split X, y de los datos
X, y = train_dataset.drop(columns="Target"), train_dataset["Target"] #Partimos el dataset de entrenamiento en X y y

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

In [7]:
# Clase para seleccionar columnas a conservar
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]  # Selecciona solo las columnas especificadas

# Columnas a conservar

#columns_to_keep =['AGRUPACION_DIASTOLICA','AGRUPACION_SISTOLICA','CANCER_MAMA_FAMILIAR','CANCER_OTRO_SITIO','CANCER_OTRO_SITIO_FAMILIAR','Cant_Fliar_CP','Cant_Fliar_riesgos','Cant_gr_flia','Cant_riesgos_flia_mean','cantidad_serv_flia','CANTIDAD_SERVICIOS','CEREBRAL','CEREBRAL_FAMILIAR','conteo_dx_diferentes','CORONARIOS','CORONARIOS_FAMILIAR','DIABETES','DIABETES_FAMILIAR','EDAD','ENFERMEDAD_RENAL','ENFERMEDAD_RENAL_FAMILIAR','ESTADO_CIVI','estrato','HIPERTENSION','HIPERTENSION_FAMILIAR','IMC','Intercepto','Intercepto_flia','MEDICAMENTOS','MEDICINA ESPECIALIZADA','MEDICINA GENERAL','OTROS_ANTECEDENTES_VASCULARES','parentesco','Pendiente','Pendiente_flia','PROGRAMA','Promedio_costo','Promedio_costo_flia','RIESGOS','TIEMPO_AFILIACION','TIEMPO_ULTIMA_CITA']

columns_to_keep = ['EDAD', 'AGRUPACION_DIASTOLICA', 'CANCER_OTRO_SITIO',
                   'CANCER_OTRO_SITIO_FAMILIAR', 'DIABETES',
                   'DIABETES_FAMILIAR', 'HIPERTENSION',
                   'HIPERTENSION_FAMILIAR']  # Especifica tus columnas a conservar

#columns_to_keep = ['EDAD', 'AGRUPACION_DIASTOLICA', 'CANCER_OTRO_SITIO',
#                    'DIABETES', 'HIPERTENSION',
#                   ]  # Especifica tus columnas a conservar





In [8]:
# Función para identificar columnas numéricas y categóricas
def identify_columns(df, columns_to_keep):
    df_filtered = df[columns_to_keep]  # Filtrar columnas
    numerical = df_filtered.select_dtypes(include=np.number).columns.tolist()  # Obtener numéricas
    categorical = df_filtered.select_dtypes(exclude=np.number).columns.tolist()  # Obtener categóricas
    return numerical, categorical

# Obtener columnas numéricas y categóricas
numerical_columns, categorical_columns = identify_columns(X_train, columns_to_keep)

# Verifica que se obtengan correctamente
print("Columnas numéricas:", numerical_columns)
print("Columnas categóricas:", categorical_columns)

Columnas numéricas: ['EDAD']
Columnas categóricas: ['AGRUPACION_DIASTOLICA', 'CANCER_OTRO_SITIO', 'CANCER_OTRO_SITIO_FAMILIAR', 'DIABETES', 'DIABETES_FAMILIAR', 'HIPERTENSION', 'HIPERTENSION_FAMILIAR']


In [9]:
# Clase personalizada para corregir el valor "2" por la moda
class ReplaceWithMode(BaseEstimator, TransformerMixin):
    def __init__(self, incorrect_value=2, columns=None):
        self.incorrect_value = incorrect_value
        self.columns = columns
        self.modes_ = {}

    def fit(self, X, y=None):
        # Calcular la moda para cada columna especificada
        for col in self.columns:
            self.modes_[col] = X[col].mode()[0]
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            # Reemplazar el valor incorrecto por la moda en las columnas especificadas
            X_copy[col] = X_copy[col].replace(self.incorrect_value, self.modes_[col])
        return X_copy

In [10]:
# Pipeline para imputar y codificar variables categóricas
categorical_pipeline = Pipeline([
    ("replace_with_mode", ReplaceWithMode(incorrect_value=2, columns=categorical_columns)),  # Reemplazar el valor 2 por la moda
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(drop='first', handle_unknown='ignore'))
])

In [11]:
# Pipeline para imputar variables numéricas
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

In [12]:
# Pipeline completo que combina todo
preprocessing_pipeline = Pipeline(steps=[
    ('column_selector', ColumnSelector(columns_to_keep)),  # Seleccionar columnas a conservar
    ('preprocessor', ColumnTransformer([
        ("numerical", numerical_pipeline, numerical_columns),  # Aplicar pipeline numérico
        ("categorical", categorical_pipeline, categorical_columns)  # Aplicar pipeline categórico
    ], remainder='drop'))  # Eliminar columnas no especificadas
])

In [13]:
preprocessing_pipeline

Pipeline(steps=[('column_selector',
                 ColumnSelector(columns=['EDAD', 'AGRUPACION_DIASTOLICA',
                                         'CANCER_OTRO_SITIO',
                                         'CANCER_OTRO_SITIO_FAMILIAR',
                                         'DIABETES', 'DIABETES_FAMILIAR',
                                         'HIPERTENSION',
                                         'HIPERTENSION_FAMILIAR'])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['E...
                                                                                            'CANCER_OTRO_SITIO_FAMILIAR',
                                                                                            'DIABETES',
                                                                                            'DIABETES_FAMILIAR',
                                                                                            'HIPERTENSION',
                                                                                            'HIPERTENSION_FAMILIAR'])),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['AGRUPACION_DIASTOLICA',
                                                   'CANCER_OTRO_SITIO',
                                                   'CANCER_OTRO_SITIO_FAMILIAR',
                                                   'DIABETES',
                                                   'DIABETES_FAMILIAR',
                                                   'HIPERTENSION',
                                                   'HIPERTENSION_FAMILIAR'])]))])

In [14]:
# Aplicar el pipeline al conjunto de datos
X_train_transformed = preprocessing_pipeline.fit_transform(X_train)
X_test_transformed = preprocessing_pipeline.transform(X_test)
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(2349, 16)
(21145, 16)


In [15]:
def objective_mlp(trial):
    # Definición de los hiperparámetros a optimizar
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50, 50), (100, 50), (100, 100)])
    activation = trial.suggest_categorical('activation', ['identity', 'logistic', 'tanh', 'relu'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam'])
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)  # Regularización L2
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive'])
    max_iter = trial.suggest_int('max_iter', 200, 1000)

    # Crear el clasificador MLP con los hiperparámetros sugeridos
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver,
                        alpha=alpha, learning_rate=learning_rate, max_iter=max_iter, random_state=42)

    # Ajustar el modelo
    mlp.fit(X_train_transformed, y_train)

    # Predecir probabilidades en el conjunto de prueba
    y_pred_prob_mlp = mlp.predict_proba(X_test_transformed)[:, 1]

    # Calcular el ROC AUC
    roc_auc = roc_auc_score(y_test, y_pred_prob_mlp)

    return roc_auc

# Crear un estudio para la optimización
study = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study.optimize(objective_mlp, n_trials=100)

# Imprimir los mejores hiperparámetros
print("Best hyperparameters:", study.best_params)
print("Best ROC AUC:", study.best_value)


[I 2024-10-02 21:32:56,508] A new study created in memory with name: no-name-5089714f-4b84-42b1-ad59-98da2c91f5ef
c:\Users\ACER\anaconda3\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
c:\Users\ACER\anaconda3\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
c:\Users\ACER\anaconda3\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50, 50) which is of type tuple.
  warnings.warn(message)
c:\Users\ACER\anaconda3\lib\site-packages\optuna\distributions.py:524: UserWarning: 

Best hyperparameters: {'hidden_layer_sizes': (100, 100), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.09601327730864784, 'learning_rate': 'adaptive', 'max_iter': 827}
Best ROC AUC: 0.6146378229151379


In [20]:
#Ajustamos el modelo con los mejores hiperparámetros encontrados
hidden_layer_sizes =  (100, 100)
activation = 'relu'
solver = 'sgd'
alpha =  0.09601327730864784  # Regularización L2
learning_rate = 'adaptive'
max_iter = 827

# Crear el clasificador MLP con los hiperparámetros sugeridos
mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver,
                    alpha=alpha, learning_rate=learning_rate, max_iter=max_iter, random_state=42)

# Ajustar el modelo
mlp.fit(X_train_transformed, y_train)

# Predecir probabilidades en el conjunto de prueba
y_pred_prob_mlp = mlp.predict_proba(X_test_transformed)[:, 1]

# Calcular el ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_prob_mlp)
print("ROC AUC:", roc_auc)

ROC AUC: 0.6146378229151379


Encontramos que el modelo arroja un ROC AUC de 0.61